In [2]:
"""
Script to calculate baseline (time 0) indicators for:
- the 5 boroughs with the highest household food insecurity
- the 5 boroughs with the lowest household food insecurity

Inputs
------
simulation_runs.xlsx             # SSD outputs per borough (NUM)
boroughs_name_NUM_match.xlsx     # mapping NUM -> borough name

Outputs
-------
table1.csv   # CSV with the values shown in Table 1
"""

import pandas as pd
import ast

# --------------------------------------------------------------------
# 1. Load data
# --------------------------------------------------------------------
# Adjust paths if needed (e.g. r"data/simulation_runs.xlsx")
sim_path = "simulation_runs.xlsx"
name_path = "boroughs_name_NUM_match.xlsx"

sim = pd.read_excel(sim_path)
names = pd.read_excel(name_path)

# merge to get borough names
df = sim.merge(names, on="NUM", how="left")

# --------------------------------------------------------------------
# 2. Helper: extract value at time 0 from dict-like columns
# --------------------------------------------------------------------
def get_time0(series):
    """
    series: a pandas Series where each cell is either
      - a Python dict, e.g. {'0.0': value0, '1.0': value1, ...}
      - or a string representation of such a dict

    returns: Series of the time 0 values.
    """
    values = []

    for x in series:
        if isinstance(x, dict):
            d = x
        elif isinstance(x, str) and x.startswith("{"):
            # safer than eval
            d = ast.literal_eval(x)
        else:
            values.append(pd.NA)
            continue

        # keys are usually '0.0', '1.0', ...
        # take the smallest key (time 0)
        try:
            # convert keys to float to sort by time
            key0 = sorted(d.keys(), key=lambda k: float(k))[0]
            values.append(d[key0])
        except Exception:
            values.append(pd.NA)

    return pd.Series(values, index=series.index)


# --------------------------------------------------------------------
# 3. Extract time-0 values for the variables used in Table 1
# --------------------------------------------------------------------
cols = {
    "HFI": "householdFoodInsecurity",
    "Availability of healthy food in local area": "availabilityOfHealthyFoodInLocalArea",
    "Supermarket and healthy food store density": "supermarketAndHealthyFoodStoreDensity",
    "Accessibility of healthy food in local area":"accessibilityOfHealthyFoodInLocalArea",
    "Accessibility and affordability of public transport": "accessibilityAndAffordabilityOfPublicTransport",
    "Household circumstances food insecurity": "householdCircumstancesFoodInsecurity",
}

for new_name, original_col in cols.items():
    df[new_name] = get_time0(df[original_col])

# --------------------------------------------------------------------
# 4. Identify 5 highest and 5 lowest HFI boroughs at time 0
# --------------------------------------------------------------------
# sort by HFI
df_sorted = df.sort_values("HFI")

# 5 lowest HFI
lowest5 = df_sorted.head(5)

# 5 highest HFI
highest5 = df_sorted.tail(5)

# combine in one table
table1 = pd.concat([highest5, lowest5], axis=0)

# select and rename columns for readability
table1 = table1[["NUM", "NOM_y", "HFI",
                 "AvailHealthy", "StoreDensity",
                 "PTaccess", "HHcirc"]].copy()

table1 = table1.rename(columns={
    "NOM_y": "Borough",
    "HFI": "HFI_t0",
    "AvailHealthy": "AvailHealthy_t0",
    "StoreDensity": "StoreDensity_t0",
    "PTaccess": "PTaccess_t0",
    "HHcirc": "HHcirc_t0",
})

# round for a nicer table (optional)
table1 = table1.round({
    "HFI_t0": 3,
    "AvailHealthy_t0": 3,
    "StoreDensity_t0": 3,
    "PTaccess_t0": 3,
    "HHcirc_t0": 3,
})

# sort so that highest-HFI rows appear first, then lowest
table1 = table1.sort_values("HFI_t0", ascending=False)

# --------------------------------------------------------------------
# 5. Save to CSV (and/or print)
# --------------------------------------------------------------------
table1.to_csv("table1.csv", index=False)
print(table1)


KeyError: "['AvailHealthy', 'StoreDensity', 'PTaccess', 'HHcirc'] not in index"